In [1]:
%load_ext autoreload
%autoreload 2

import os
import sys
os.environ['UTILS_PATH'] = r'/home/nkrokhmal/Desktop/Git/python-utils-ak'
os.environ['PROJECT_PATH'] = r'/home/nkrokhmal/Desktop/umalat'
sys.path.append(os.environ['PROJECT_PATH'])
from app.interactive_imports import *


import math

In [2]:
df = pd.read_excel(r'/home/nkrokhmal/Downloads/Остатки  17,12,2020..xlsx')

In [8]:
sku_names = df.iloc[3].dropna().to_list()[1:-1]

In [13]:
skus = db.session.query(SKU).all()
skus_db_names = [x.name for x in skus]

In [16]:
skus_db_not = [x for x in sku_names if x not in skus_db_names]

In [17]:
skus_db_not

['Кавказский "Умалат" (Перекресток), 45%, кг, в/у',
 'Кавказский "Умалат" (Окей), 45%, кг, в/у',
 'Кавказский "Умалат" (Тандер), 45%, кг , в/у',
 'Кавказский "Умалат" (Метро), 45%, кг, в/у',
 'Кавказский "Умалат", 45%, 0,37 кг, в/у',
 'Кавказский "Глобус", 45%, кг, в/у',
 'Кавказский "Красная птица", 45%, 0,37 кг, в/у',
 'Кавказский "Умалат", 45%, кг, в/у',
 'Сыр Черкесский "Умалат", 45%, 0,28 кг, т/ф',
 'Сыр Черкесский "Умалат", 45%, кг, т/ф, ВЕС',
 'Сыр Черкесский "Умалат" (БИЛЛА), 45%, т/ф, ВЕС',
 'Сыр Черкесский "Умалат" (ДИКСИ), 45%, т/ф, ВЕС',
 'Свели-Квели "Умалат", 30%, 0,37 кг, в/у',
 'Четук "Умалат", 45%, 0,37 кг, в/у',
 'Качорикотта "Unagrande", 45%, 0,37 кг, в/у',
 'Рикотта "Pretto" (зернистая), 30%, 0,37 кг, в/у',
 'Сулугуни палочки "Красная птица", 45%, 0,12 кг, т/ф',
 'Сулугуни без лактозы "ВкусВилл", 45%, 0,2 кг, т/ф',
 'Моцарелла палочки "Красная птица", 45%, 0,12 кг, т/ф',
 'Моцарелла для пиццы "Красная птица", 45%, 0,28 кг, т/ф',
 'Моцарелла для бутербродов "Aventino

In [6]:
COLOURS = {
    'Для пиццы': '#E5B7B6',
    'Моцарелла': '#DAE5F1',
    'Фиор Ди Латте': '#CBC0D9',
    'Чильеджина': '#E5DFEC',
    'Качокавалло': '#F1DADA',
    'Сулугуни': '#F1DADA',
    'Default': '#FFFFFF'
}

In [159]:
def generate_constructor_df(df):
    values = []
    cur_id = 1
    for boiling_id, boiling_grp in df.groupby('boiling_id'):
        boiling_grp['weight'] = boiling_grp['sku'].apply(lambda x: x.weight_netto)
        boiling_grp = boiling_grp.sort_values(by='weight')
        # create sku_plan as dict
        sku_plan = OrderedDict() # {sku: kg}
        for i, row in boiling_grp[['sku', 'plan']].iterrows():
            if row['sku'] not in sku_plan:
                sku_plan[row['sku']] = 0
            sku_plan[row['sku']] += row['plan']

        total_kg = sum(sku_plan.values())

        # round to get full
        boiling_type = 'salt' if str(cast_boiling(boiling_id).lines.name) == 'Пицца чиз' else 'water'
        # todo: take from db
        volume = 1000 if boiling_type == 'water' else 850

        total_kg = custom_round(total_kg, volume, rounding='ceil')
        n_boilings = int(total_kg / volume)

        for i in range(n_boilings):
            cur_kg = volume

            boiling_contents = OrderedDict()
            for sku, kg in list(sku_plan.items()):
                boil_kg = min(cur_kg, sku_plan[sku])

                sku_plan[sku] -= boil_kg
                cur_kg -= boil_kg

                if sku not in boiling_contents:
                    boiling_contents[sku] = 0
                boiling_contents[sku] += boil_kg

                if cur_kg == 0:
                    break

            if cur_kg != 0:
                print('Non-zero', sku, kg, cur_kg)
                sku = [k for k, v in boiling_contents.items() if v != 0][0]
                boiling_contents[sku] += cur_kg

            for sku, kg in boiling_contents.items():
                values.append([cur_id, cast_boiling(boiling_id), sku, kg])
            cur_id += 1

    boiling_plan_df = pd.DataFrame(values, columns=['id', 'boiling', 'sku', 'kg'])
    boiling_plan_df = boiling_plan_df[boiling_plan_df['kg'] != 0]
    return boiling_plan_df


def generate_full_constructor_df(boiling_plan_df):
    # create dataframe for samples
    df = boiling_plan_df.copy()
    df['name'] = df['sku'].apply(lambda sku: sku.name)
    # todo: make properly
    df['boiling_name'] = df['boiling'].apply(lambda b: '{} {} {}'.format(b.percent, b.ferment, '' if b.is_lactose else 'без лактозы'))
    # todo: make properly
    df['boiling_volume'] = np.where(df['boiling_name'].str.contains('2.7'), 850, 1000)
    df['form_factor'] = df['sku'].apply(lambda sku: sku.form_factor.name)
    df['boiling_id'] = df['boiling'].apply(lambda b: b.id)
    df = df[['id', 'boiling_id', 'boiling_name','boiling_volume','form_factor', 'name', 'kg']]
    df = df.sort_values(by=['boiling_id', 'id', 'boiling_name', 'form_factor', 'name'])
    return df.reset_index(drop=True)

def generate_empty_sku():
    values = session.query(SKU).all()
    skus = pd.DataFrame(values, columns=['sku'])
    skus['boiling'] = skus['sku'].apply(lambda sku: sku.boilings[0])
    skus['id'] = ''
    skus['kg'] = 0
    skus = skus[['id', 'boiling', 'sku', 'kg']]
    return skus


def draw_constructor(df, file_name):
    wb = init_sheets('Соль', 'Вода')

    skus = session.query(SKU).all()
    data_sku = {'Вода': [x.name for x in skus if x.boilings[0].cheese_types.name == 'Вода'],
                'Соль': [x.name for x in skus if x.boilings[0].cheese_types.name == 'Соль']}

    for sheet_name in ['Соль', 'Вода']:
        sku_names = data_sku[sheet_name]
        sheet = wb[sheet_name]
        for i, v in enumerate([15, 15, 15, 15, 15, 50, 15], 1):
            sheet.column_dimensions[get_column_letter(i)].width = v

        cur_i = 1
        draw_row(sheet, cur_i,
                 ['id варки', "Номер варки", 'Тип варки', 'Объем варки', 'Форм фактор', 'SKU', 'КГ', 'Разделитель'],
                 font_size=8)
        cur_i += 1

        values = []
        df_filter = df[df['name'].isin(sku_names)].copy()
        for id, grp in df_filter.groupby('id'):
            for i, row in grp.iterrows():
                v = []
                v += list(row.values)
                v += ['']
                values.append(v)

            # add separator
            values.append(['-'] * (len(df_filter.columns) + 1))

        # todo: column names to config
        for v in values:
            sheet.column_dimensions['J'].hidden = True
            sheet.column_dimensions['K'].hidden = True
            sheet.column_dimensions['L'].hidden = True
            sheet.column_dimensions['M'].hidden = True

            formula_remains = '=IF({0}{1} - {0}{2} = 0, "", {0}{1} - {0}{2})'.format('M', cur_i, cur_i - 1)
            formula_calc = '=IF({0}{3} = "-", -{1}{4},{2}{3})'.format('I', 'D', 'G', cur_i, cur_i - 1)
            formula_remains_cumsum = '=IF({0}{2} = "-", SUM({1}$2:J{2}), 0)'.format('I', 'J', cur_i, cur_i - 1)
            formula_delimiter_int = '=IF({0}{1}="-",1,0)'.format('I', cur_i)
            formula_zeros = '=IF({0}{2} = 0, {1}{3}, {0}{2})'.format('K', 'M', cur_i, cur_i - 1)

            v.insert(-1, formula_remains)
            v.append(formula_calc)
            v.append(formula_remains_cumsum)
            v.append(formula_delimiter_int)
            v.append(formula_zeros)

            colour = get_colour_by_name(v[5], skus)
            draw_row(sheet, cur_i, v, font_size=8, color=colour)
            cur_i += 1

        cur_i += 1
        skus_df = generate_full_constructor_df(generate_empty_sku())
        skus_df = skus_df[skus_df['name'].isin(sku_names)]
        values = []
        for boiling_name, grp1 in skus_df.groupby('boiling_name'):
            for form_factor, grp2 in grp1.groupby('form_factor'):
                for i, row in grp2.iterrows():
                    values.append(list(row.values))

        for v in values:
            colour = get_colour_by_name(v[5], skus)
            draw_row(sheet, cur_i, v, font_size=8, color=colour)
            cur_i += 1

    path = '{}/{}.xlsx'.format('/home/nkrokhmal/Desktop/umalat/app/data/tmp', os.path.splitext(file_name)[0])
    link = '{}/{}.xlsx'.format('/home/nkrokhmal/Desktop/umalat/app/data/tmp', os.path.splitext(file_name)[0])
    wb.save(path)
    return link


def get_colour_by_name(sku_name, skus):
    sku = [x for x in skus if x.name == sku_name]
    if len(sku) > 0:
        return COLOURS[sku[0].form_factor.name]
    else:
        return COLOURS['Default']


In [160]:
path = '/home/nkrokhmal/Downloads/11.12.2020.csv'

In [255]:
def f(path):
    file_name = '11.12.2020.csv'
    df = pd.read_csv(path)
    df['sku'] = df['sku'].apply(cast_sku)
    df = df.replace(to_replace='None', value=np.nan).dropna()
    df['boiling_id'] = df['sku'].apply(lambda x: x.boilings[0].id)
    df['sku_id'] = df['sku'].apply(lambda x: x.id)
    df['plan'] = df['plan'].apply(lambda x: -float(x))
    df = df[['boiling_id', 'sku_id', 'plan']]
    df['sku'] = df['sku_id'].apply(cast_sku)
    boiling_plan_df = generate_constructor_df(df)
    full_plan = generate_full_constructor_df(boiling_plan_df)
    link = draw_constructor(full_plan, file_name)
    return boiling_plan_df

In [256]:
f(path)

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  
/home/nkrokhmal/.local/lib/python3.6/site-packages/pandas/core/indexing.py:671: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self._setitem_with_indexer(indexer, value)
/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a

{'boiling_id': 1, 'sku_id': 39, 'plan': 850, 'sku': <SKU 39>, 'weight': 280.0}
{'boiling_id': 1, 'sku_id': 39, 'plan': 850, 'sku': <SKU 39>, 'weight': 280.0}
{'boiling_id': 1, 'sku_id': 39, 'plan': 850, 'sku': <SKU 39>, 'weight': 280.0}
{'boiling_id': 1, 'sku_id': 39, 'plan': 850, 'sku': <SKU 39>, 'weight': 280.0}
{'boiling_id': 1, 'sku_id': 39, 'plan': 850, 'sku': <SKU 39>, 'weight': 280.0}
{'boiling_id': 1, 'sku_id': 37, 'plan': 850, 'sku': <SKU 37>, 'weight': 120.0}
{'boiling_id': 1, 'sku_id': 34, 'plan': 850, 'sku': <SKU 34>, 'weight': 370.0}
{'boiling_id': 1, 'sku_id': 14, 'plan': 850, 'sku': <SKU 14>, 'weight': 460.0}
{'boiling_id': 1, 'sku_id': 20, 'plan': 770.4, 'sku': <SKU 20>, 'weight': 120.0}
{'boiling_id': 1, 'sku_id': 37, 'plan': 79.60000000000002, 'sku': <SKU 37>, 'weight': 120.0}
{'boiling_id': 1, 'sku_id': 35, 'plan': 783.84, 'sku': <SKU 35>, 'weight': 280.0}
{'boiling_id': 1, 'sku_id': 17, 'plan': 66.15999999999997, 'sku': <SKU 17>, 'weight': 280.0}
{'boiling_id': 1, '

,id,boiling,sku,kg
0,0,<Boiling 1>,<SKU 39>,850.00
1,1,<Boiling 1>,<SKU 39>,850.00
2,2,<Boiling 1>,<SKU 39>,850.00
3,3,<Boiling 1>,<SKU 39>,850.00
4,4,<Boiling 1>,<SKU 39>,850.00
5,5,<Boiling 1>,<SKU 37>,850.00
6,6,<Boiling 1>,<SKU 34>,850.00
7,7,<Boiling 1>,<SKU 14>,850.00
8,8,<Boiling 1>,<SKU 20>,770.40
9,8,<Boiling 1>,<SKU 37>,79.60


In [239]:
res

[{'boiling_id': 1,
  'sku_id': 39,
  'plan': 850,
  'sku': <SKU 39>,
  'weight': 280.0},
 {'boiling_id': 1,
  'sku_id': 39,
  'plan': 850,
  'sku': <SKU 39>,
  'weight': 280.0},
 {'boiling_id': 1,
  'sku_id': 39,
  'plan': 850,
  'sku': <SKU 39>,
  'weight': 280.0},
 {'boiling_id': 1,
  'sku_id': 39,
  'plan': 850,
  'sku': <SKU 39>,
  'weight': 280.0},
 {'boiling_id': 1,
  'sku_id': 39,
  'plan': 850,
  'sku': <SKU 39>,
  'weight': 280.0},
 {'boiling_id': 1,
  'sku_id': 37,
  'plan': 850,
  'sku': <SKU 37>,
  'weight': 120.0},
 {'boiling_id': 1,
  'sku_id': 34,
  'plan': 850,
  'sku': <SKU 34>,
  'weight': 370.0},
 {'boiling_id': 1,
  'sku_id': 14,
  'plan': 850,
  'sku': <SKU 14>,
  'weight': 460.0},
 [{'boiling_id': 1,
   'sku_id': 20,
   'plan': 770.4,
   'sku': <SKU 20>,
   'weight': 120.0},
  {'boiling_id': 1,
   'sku_id': 37,
   'plan': 79.60000000000002,
   'sku': <SKU 37>,
   'weight': 120.0}],
 [{'boiling_id': 1,
   'sku_id': 35,
   'plan': 783.84,
   'sku': <SKU 35>,
   'wei

In [257]:
def generate_constructor_df(df):
    values = []
    cur_id = 1
    result = []
    for boiling_id, boiling_grp in df.groupby('boiling_id'):
        boiling_grp['weight'] = boiling_grp['sku'].apply(lambda x: x.weight_netto)
        boiling_volume = boiling_grp['sku'].iloc[0].boilings[0].cheese_types.output
        
        boiling = 1
        while boiling is not None:
            boiling_grp, boiling = iteration(boiling_grp, boiling_volume)
            if boiling is not None:
                result.append(boiling)
        if (boiling_grp['plan'].sum() > 0):
            result.append(boiling_grp.to_dict('records'))
    
    full_result = []
    for i, boiling in enumerate(result):
        for boiling_element in boiling:
            full_result.append({
                'id': i,
                'boiling': boiling_element['sku'].boilings[0],
                'sku': boiling_element['sku'],
                'kg': boiling_element['plan']
            })
    return pd.DataFrame(full_result)

In [235]:
def iteration(df, volume):
    full_boilings = df[df['plan'] > volume]
    if full_boilings.shape[0] > 0:
        new_boiling = full_boilings.iloc[0].to_dict()
        new_boiling['plan'] = volume
        df['plan'].loc[df['sku'] == new_boiling['sku']] -= volume
        return df, [new_boiling]
    
    for weight, boiling_grp in df.groupby('weight'):
        if boiling_grp['plan'].sum() > volume:
            boiling_grp = boiling_grp.sort_values(by='plan', ascending=False).to_dict('records') 
            new_boiling = []
            cur_weight = 0
            for b_grp in boiling_grp:
                if cur_weight + b_grp['plan'] < volume:
                    new_boiling.append(b_grp)
                    df['plan'].loc[df['sku'] == b_grp['sku']] = 0
                    cur_weight += b_grp['plan']  
                else:
                    b_grp['plan'] = volume - cur_weight
                    new_boiling.append(b_grp)
                    df['plan'].loc[df['sku'] == b_grp['sku']] -= volume - cur_weight
                    return df, new_boiling
    df = df[df['plan'] > 0]
    new_boiling = []
    cur_weight = 0
    for data in df.sort_values(by='weight').to_dict('records'):
        if cur_weight + data['plan'] < volume:
            new_boiling.append(data)
            df['plan'].loc[df['sku'] == data['sku']] = 0
            cur_weight += data['plan']  
        else:
            data['plan'] = volume - cur_weight
            new_boiling.append(data)
            df['plan'].loc[df['sku'] == data['sku']] -= volume - cur_weight
            return df, new_boiling
    return df, None
        
   